In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import plotly.graph_objects as go

In [6]:
data = pd.read_csv('data/Arka.csv', delimiter=';')
data.head()

,Unnamed: 0,Date,Goals,Possesion_In_%,xG,Shots,Shots_On_Target,Shots_Off_Target,Shots_Blocked,Shots_Outside_Box,...,Corners,Offsides,Yellow_Cards,Red_Cards,Penalties,Free_Kicks,Goal_Kicks,Throw_Ins,Saves,Crosses
0,Motor,20.07.2025,0.0,40.0,"0,38",10.0,1.0,5.0,4.0,4.0,...,8.0,3.0,4.0,0.0,0.0,17.0,12.0,16.0,3.0,22.0
1,Radomiak,25.07.2025,1.0,49.0,"1,08",10.0,4.0,5.0,1.0,3.0,...,5.0,3.0,1.0,0.0,0.0,14.0,12.0,21.0,2.0,16.0
2,Legia,03.08.2025,0.0,37.0,"0,56",6.0,0.0,3.0,3.0,1.0,...,4.0,1.0,3.0,0.0,0.0,20.0,11.0,14.0,3.0,18.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data = data.rename(columns={'Unnamed: 0': 'Team'})
data = data.set_index('Team')
data['xG'] = data['xG'].str.replace(',', '.').astype('float64')
data.head()

,Date,Goals,Possesion_In_%,xG,Shots,Shots_On_Target,Shots_Off_Target,Shots_Blocked,Shots_Outside_Box,Shots_Inside_Box,...,Corners,Offsides,Yellow_Cards,Red_Cards,Penalties,Free_Kicks,Goal_Kicks,Throw_Ins,Saves,Crosses
Team,,,,,,,,,,,,,,,,,,,,,
Motor,20.07.2025,0.0,40.0,0.38,10.0,1.0,5.0,4.0,4.0,6.0,...,8.0,3.0,4.0,0.0,0.0,17.0,12.0,16.0,3.0,22.0
Radomiak,25.07.2025,1.0,49.0,1.08,10.0,4.0,5.0,1.0,3.0,7.0,...,5.0,3.0,1.0,0.0,0.0,14.0,12.0,21.0,2.0,16.0
Legia,03.08.2025,0.0,37.0,0.56,6.0,0.0,3.0,3.0,1.0,5.0,...,4.0,1.0,3.0,0.0,0.0,20.0,11.0,14.0,3.0,18.0
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Create the figure
from torch import layout


fig = go.Figure()

# List of metrics to plot
metrics = [
    'Goals', 'Possesion_In_%', 'xG', 'Shots', 'Shots_On_Target',
       'Shots_Off_Target', 'Shots_Blocked', 'Shots_Outside_Box',
       'Shots_Inside_Box', 'Passes', 'Tackles', 'Fouls', 'Corners', 'Offsides',
       'Yellow_Cards', 'Red_Cards', 'Penalties', 'Free_Kicks', 'Goal_Kicks',
       'Throw_Ins', 'Saves', 'Crosses']

# Add traces for each metric
for metric in metrics:
    fig.add_trace(go.Bar(x=data.index, y=data[metric], name=metric))


# Define visibility matrix
n = len(metrics)
visibility_matrix = {
    "None": [False] * n,
    "Goals and xG": [metric in ['Goals', 'xG'] for metric in metrics],
    "Possesion": [metric in ['Possesion_In_%'] for metric in metrics],
    "Passes": [metric in ['Passes'] for metric in metrics],
    "Attack": [metric in ['Shots', 'Shots_On_Target', 'Shots_Off_Target', 'Shots_Inside_Box', 'Shots_Outside_Box'] for metric in metrics],
    "Defense": [metric in ['Tackles', 'Saves', 'Fouls'] for metric in metrics],
    "Cards": [metric in ['Yellow_Cards', 'Red_Cards'] for metric in metrics],
    "All": [True] * n
}

# Create buttons
buttons = [
    dict(label=label,
         method="update",
         args=[{"visible": visibility},
               {"title": f"Arka Match Stats: {label}"}])
    for label, visibility in visibility_matrix.items()
]

# Add dropdown to figure
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=1.1,
        y=1.15
    )],
    title="Arka Match Stats",
    xaxis_title="Match",
    yaxis_title="Value",
    barmode='group'
)

fig.show()
fig.write_html("Arka_stats.html")